In [2]:
from langgraph.graph import StateGraph,START,END
from typing import Annotated, TypedDict
from langchain_core.messages import BaseMessage,HumanMessage
from langchain_cerebras import ChatCerebras
from dotenv import load_dotenv
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import InMemorySaver

load_dotenv()
llm=ChatCerebras(model='qwen-3-32b')
class JokeState(TypedDict):
    topic:str
    joke:str
    explanation:str

def generate_joke(state:JokeState):
    prompt=f'generate a joke on the topic {state['topic']}'
    response=llm.invoke(prompt).content
    return {'joke':response}

def generate_explanation(state:JokeState):
    prompt=f'write an explanation for the joke {state['joke']}'
    response=llm.invoke(prompt).content
    return {'explanation':response}

graph=StateGraph(JokeState)

graph.add_node('generate_joke',generate_joke)
graph.add_node('generate_explanation',generate_explanation)
graph.add_edge(START,'generate_joke')
graph.add_edge('generate_explanation',END)

checkpointer=InMemorySaver()

workflow=graph.compile(checkpointer=checkpointer)

config1={"configurable":{"thread_id":"1"}}
result=workflow.invoke({'topic':'pizza'},config=config1)

#get the last state value
print(workflow.get_state(config1))

#getting all the state values 
print(list(workflow.get_state_history(config1)))


StateSnapshot(values={'topic': 'pizza', 'joke': '<think>\nOkay, the user wants a pizza joke. Let me think about pizza-related topics. Maybe something with ingredients? Like how a pizza is made? Or maybe a play on words with "pizza" itself. Hmm. Let\'s see. There\'s "Why did the pizza go to the doctor?" Maybe something about feeling burned? Or maybe a topping joke. Oh, "What did the pizza say when it was late?" "I\'m stuffed!" Wait, that\'s a classic. Maybe I need something original. How about combining elements like different toppings? Maybe a pun with "cheese" or "crust"? Or maybe a math-related joke? Like slices and angles? Like a pizza pie chart? Okay, here\'s an idea: Why don\'t pizzas ever get cold? Because they always have a warm crust! Hmm, crust is the key word here. Or maybe a joke about pepperoni. Why did the pepperoni walk? To get to the tomato sauce. Wait, that\'s not great. Let me think again. Here\'s one: What do you call a fake pizza? An impasta! Because "impasta" sounds

In [3]:
config2={"configurable":{"thread_id":"2"}}
workflow.invoke({'topic':'pasta'},config=config2)

{'topic': 'pasta',
 'joke': '<think>\nOkay, I need to come up with a joke about pasta. Let\'s see... Pasta is a broad topic, so maybe I can focus on a specific type or a situation involving pasta. Common types are spaghetti, lasagna, ravioli, etc. Maybe play on words with "pasta" sounding like "past a" something? Or maybe a pun related to cooking.\n\nLet me think of some words related to pasta: noodles, sauce, boil, dough, al dente, fork, plate,意大利面 (though that\'s Chinese for Italian noodles, maybe not useful here). Maybe a play on "I\'talian" pasta? Not sure.\n\nAnother angle: why did the pasta...? Like why did the spaghetti get in trouble? Because it was too long? Or maybe it was twirling in the pan? Hmm. Or maybe a joke about pasta being messy. \n\nWhat about combining pasta with another concept? Like a pun involving "knot" since spaghetti can get knotted. Oh, like "Why don\'t pasta ever get cold? Because they always wear a noodle!" Wait, that\'s not great. Maybe something with "tw

In [4]:
workflow.get_state(config2)

StateSnapshot(values={'topic': 'pasta', 'joke': '<think>\nOkay, I need to come up with a joke about pasta. Let\'s see... Pasta is a broad topic, so maybe I can focus on a specific type or a situation involving pasta. Common types are spaghetti, lasagna, ravioli, etc. Maybe play on words with "pasta" sounding like "past a" something? Or maybe a pun related to cooking.\n\nLet me think of some words related to pasta: noodles, sauce, boil, dough, al dente, fork, plate,意大利面 (though that\'s Chinese for Italian noodles, maybe not useful here). Maybe a play on "I\'talian" pasta? Not sure.\n\nAnother angle: why did the pasta...? Like why did the spaghetti get in trouble? Because it was too long? Or maybe it was twirling in the pan? Hmm. Or maybe a joke about pasta being messy. \n\nWhat about combining pasta with another concept? Like a pun involving "knot" since spaghetti can get knotted. Oh, like "Why don\'t pasta ever get cold? Because they always wear a noodle!" Wait, that\'s not great. Mayb

In [10]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': '<think>\nOkay, the user wants a pizza joke. Let me think about pizza-related topics. Maybe something with ingredients? Like how a pizza is made? Or maybe a play on words with "pizza" itself. Hmm. Let\'s see. There\'s "Why did the pizza go to the doctor?" Maybe something about feeling burned? Or maybe a topping joke. Oh, "What did the pizza say when it was late?" "I\'m stuffed!" Wait, that\'s a classic. Maybe I need something original. How about combining elements like different toppings? Maybe a pun with "cheese" or "crust"? Or maybe a math-related joke? Like slices and angles? Like a pizza pie chart? Okay, here\'s an idea: Why don\'t pizzas ever get cold? Because they always have a warm crust! Hmm, crust is the key word here. Or maybe a joke about pepperoni. Why did the pepperoni walk? To get to the tomato sauce. Wait, that\'s not great. Let me think again. Here\'s one: What do you call a fake pizza? An impasta! Because "impasta" sound

time travel

In [11]:
workflow.get_state({"configurable":{"thread_id":"1","checkpoint_id":"1f0e4b94-a9e3-670c-8001-431ee59e9b47"}})

StateSnapshot(values={}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f0e4b94-a9e3-670c-8001-431ee59e9b47'}}, metadata=None, created_at=None, parent_config=None, tasks=(), interrupts=())

In [12]:
workflow.invoke(None,{"configurable":{"thread_id":"1","checkpoint_id":"1f0e4b91-464f-6d8f-8000-ee8b09113480"}})

{'topic': 'pizza',
 'joke': '<think>\nOkay, the user wants a joke about pizza. Let me think... Pizza is a common topic, so I need something relatable. Maybe play on a common situation, like a pizza delivery mix-up. Or maybe a pun with toppings? Let\'s see. How about a math-related pizza joke? Like dividing a pizza into slices but ending up with a funny result. Wait, maybe a pun with "slice" or "cheese." Or maybe a joke about a pizza restaurant with a funny name. Let me try something with the pizza ingredients.\n\nWhat about a play on words with pepperoni? Like "pepperoni" and "for you"? No, that doesn\'t make sense. How about a joke where the pizza is trying to tell a joke but can\'t. Wait, maybe the pepperoni has a pepper problem? Not funny enough.\n\nMaybe a delivery joke. Like the delivery guy brings the pizza but it\'s a different shape. Triangle, square, but then another shape? Wait, pizza is usually a circle, cut into triangles. Maybe the pizza arrives as a square? But that\'s no

In [13]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': '<think>\nOkay, the user wants a joke about pizza. Let me think... Pizza is a common topic, so I need something relatable. Maybe play on a common situation, like a pizza delivery mix-up. Or maybe a pun with toppings? Let\'s see. How about a math-related pizza joke? Like dividing a pizza into slices but ending up with a funny result. Wait, maybe a pun with "slice" or "cheese." Or maybe a joke about a pizza restaurant with a funny name. Let me try something with the pizza ingredients.\n\nWhat about a play on words with pepperoni? Like "pepperoni" and "for you"? No, that doesn\'t make sense. How about a joke where the pizza is trying to tell a joke but can\'t. Wait, maybe the pepperoni has a pepper problem? Not funny enough.\n\nMaybe a delivery joke. Like the delivery guy brings the pizza but it\'s a different shape. Triangle, square, but then another shape? Wait, pizza is usually a circle, cut into triangles. Maybe the pizza arrives as a s

update state

In [17]:
workflow.update_state({"configurable":{"thread_id":"1","checkpoint_id":"1f0e4b91-464f-6d8f-8000-ee8b09113480","checkpoint_ns":""}},{"topic":"samosa"})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0e4bce-b5ef-6e2d-8001-7adcac57fd35'}}

In [18]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0e4bce-b5ef-6e2d-8001-7adcac57fd35'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2025-12-29T13:47:30.234094+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0e4b91-464f-6d8f-8000-ee8b09113480'}}, tasks=(PregelTask(id='7d255100-1e35-3bef-9ec9-3c53ffec5e00', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': '<think>\nOkay, the user wants a joke about pizza. Let me think... Pizza is a common topic, so I need something relatable. Maybe play on a common situation, like a pizza delivery mix-up. Or maybe a pun with toppings? Let\'s see. How about a math-related pizza joke? Like dividing a pizza into slices but ending up with a funny resu

In [19]:
workflow.invoke(None,{"configurable":{"thread_id":"1","checkpoint_id":"1f0e4bce-b5ef-6e2d-8001-7adcac57fd35"}})

{'topic': 'samosa',
 'joke': '<think>\nOkay, I need to come up with a joke about samosas. Let me start by thinking about what samosas are. They\'re a popular Indian snack, basically a fried or baked pastry with a savory filling. They\'re usually triangular, made with a chickpea flour dough, and filled with spiced potatoes, onions, maybe some peas. Sometimes they\'re served with chutney.\n\nNow, how can I make a joke out of this? Maybe play on words with "samosa"? Or maybe something about the ingredients. The shape is triangular, so maybe a math pun? Like triangles and samosas? Hmm. Or maybe the ingredients like potatoes and spices.\n\nWait, there\'s also a popular game called "Samosa Stack" from the Indian TV show "Comedy Nights Bachao". Maybe reference that? But I don\'t know if that\'s widely known. Maybe better to stick to common elements.\n\nLet\'s try the triangle angle. Maybe something like: Why did the samosa go to school? Because it wanted to improve its triangle! But that\'s a

In [20]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa', 'joke': '<think>\nOkay, I need to come up with a joke about samosas. Let me start by thinking about what samosas are. They\'re a popular Indian snack, basically a fried or baked pastry with a savory filling. They\'re usually triangular, made with a chickpea flour dough, and filled with spiced potatoes, onions, maybe some peas. Sometimes they\'re served with chutney.\n\nNow, how can I make a joke out of this? Maybe play on words with "samosa"? Or maybe something about the ingredients. The shape is triangular, so maybe a math pun? Like triangles and samosas? Hmm. Or maybe the ingredients like potatoes and spices.\n\nWait, there\'s also a popular game called "Samosa Stack" from the Indian TV show "Comedy Nights Bachao". Maybe reference that? But I don\'t know if that\'s widely known. Maybe better to stick to common elements.\n\nLet\'s try the triangle angle. Maybe something like: Why did the samosa go to school? Because it wanted to improve its tr